In [1]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
%run -i ../../zkstats/core.py

In [4]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [5]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))


dummy_data = np.round(np.random.uniform(1,10,len(data)),1)
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
dummy_theory_output = torch.exp(torch.mean(torch.log(dummy_data_tensor)))



In [6]:
# Verifier/ data consumer side:
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)

    def forward(self,X):
        return (torch.abs((torch.log(self.w)*X.size()[1])-torch.sum(torch.log(X)))<=X.size()[1]*torch.log(torch.tensor(1.01)), self.w)

verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_85593/1255054198.py:8: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return (torch.abs((torch.log(self.w)*X.size()[1])-torch.sum(torch.log(X)))<=X.size()[1]*torch.log(torch.tensor(1.01)), self.w)


In [10]:
# prover calculates settings, send to verifier
theory_output = torch.exp(torch.mean(torch.log(data_tensor)))
print("Theory output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)

    def forward(self,X):
        return (torch.abs((torch.log(self.w)*X.size()[1])-torch.sum(torch.log(X)))<=X.size()[1]*torch.log(torch.tensor(1.01)), self.w)


prover_gen_settings([data_path], comb_data_path, prover_model,prover_model_path, "default", "resources", settings_path)

Theory output:  tensor(47.6981)
==== Generate & Calibrate Setting ====


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_85593/4282877004.py:11: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return (torch.abs((torch.log(self.w)*X.size()[1])-torch.sum(torch.log(X)))<=X.size()[1]*torch.log(torch.tensor(1.01)), self.w)


scale:  default
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":8,"param_scale":9,"scale_rebase_multiplier":10,"lookup_range":[-3042,45056],"logrows":16,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":911,"total_const_size":5,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,9],"model_input_scales":[8],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"Ln":{"scale":256.0}},{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


In [11]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 6.590507984161377 seconds
Theory output:  tensor(47.6981)
!@# compiled_model exists? True
!@# compiled_model exists? True
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 47.697265625
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[12773529019820117503, 7223165032241168505, 6388811546753741567, 1325363978837834469], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [2305703999912675162, 13621332081612358697, 8265086842419367025, 2630935785179709972]]], 'proof': '1e54e416087fbd62eafcb143e103a4b71218957506104635e46bdd042a548c382af342b73f0639f4842509dd984fb77f99b686e49cab60cc7e6e0ac1a94cedfc2b3bed1798a9e649d8c59646461be2d30a166bd1446e45833e69361d86971d2b0caf01f79294cb2b2888c68b354d97775ca6088935d9000305a9ac58ef0523b70d1599700feccbfdb0344f375a2c7a600f23584546d49c6d0c60028f508ee7fb0c99429b2a9bdb83750047b5757fac406e2b4ab135f6307f8dc08ade374165f4176c5eaddd1e265ddbbef9525ff016cda3d4de8fd0d71627aee16a3fe20273c0260886591fdde985c1f12d4cca75d9b738e01ea8b652d1ac6760ab6b1e5991742e97b3aea91bb0336eef15ad0daabba352dc576c1c799e094e5fe15b8d5e81fb1d4e39b86884d491ec27fff9bd0ffa503adff6343510e7b5dd3c6dff093bacc02e4353a98b109f8ce477a9abaf7d8ac50bb4e785b39a9dfa7f0e3da22d1d264

In [9]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path)

num_inputs:  1
prf instances:  [[[12773529019820117503, 7223165032241168505, 6388811546753741567, 1325363978837834469], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [14148586052410868650, 7343294927740416096, 1182455714804702497, 77845129035718797]]]
proof boolean:  1.0
proof result 1 : 47.69921875
verified
